In [1]:
# !pip install z3

In [1]:
!pip install z3-solver


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

In [140]:
from typing import TypedDict, Dict, Any, List, Optional
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph
import json
from z3 import *
import re

class LogicalState(TypedDict):
    input: str
    logical_statements: List[str]
    predicates: Dict[str, Any]  # Store predicate functions
    constants: Dict[str, Any]   # Store constants
    symbolic_representation: Any
    status: str
    error: Optional[str]
    result: Optional[bool]
    context: Dict[str, Any]
    messages: List[BaseMessage]
    reasoning: List[str]

def extract_predicates(statements: List[str]) -> List[str]:
    """Extract predicate names from logical statements"""
    predicates = set()
    pattern = r'([A-Z][a-zA-Z]*)\('
    
    for stmt in statements:
        matches = re.findall(pattern, stmt)
        predicates.update(matches)
    
    return list(predicates)

def extract_constants(statements: List[str]) -> List[str]:
    """Extract constant names from logical statements"""
    constants = set()
    pattern = r'\(([a-z][a-zA-Z]*)\)'
    
    for stmt in statements:
        matches = re.findall(pattern, stmt)
        constants.update(matches)
    
    return list(constants)

def natural_to_logical(state: LogicalState) -> LogicalState:
    """Convert natural language to logical statements"""
    llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
    
    prompt = SystemMessage(content="""
    #Role
    - You are a logical reasoning agent.
    #Purpose
    - Convert natural language statements into formal logical statements.
    - Use standard logical notation and follow these rules:
    #Rules
        1. Logical symbols:
          - Universal quantifier: ∀x
          - Existential quantifier: ∃x
          - Implication: →
          - Conjunction: ∧
          - Disjunction: ∨
          - Negation: ¬
          - Therefore: ∴

        2. Format rules:
          - Number each statement
          - Predicates must start with uppercase (e.g., Human(x))
          - Variables and constants must start with lowercase
          - Mark the conclusion with ∴
          - Keep statements purely logical (no explanatory notes)

        3. Translation principles:
          - Use quantifiers only when expressing properties over a domain
          - Use simple implications for direct cause-effect relationships
          - Preserve the logical structure of the original statement
          - Maintain logical consistency between premises and conclusion
          - Express negations explicitly using ¬
          
        4. Special Cases and Common Patterns:
          a) "Only A are B":
              - Means "if something is B, then it must be A"
              - Write as: "∀x (B(x) → A(x))"
              Example: "Only mammals have fur" → "∀x (HasFur(x) → Mammal(x))"
          
          b) Double negation:
              - "not impossible" = "possible"
              - Write as direct positive when possible
              Example: "If not impossible then possible" → "¬Impossible(x) → Possible(x)"
          
          c) Mutual exclusivity:
              - "cannot be both A and B" = "nothing is both A and B"
              - Write as: "∀x ¬(A(x) ∧ B(x))"
              Example: "Nothing can be both solid and liquid" → "∀x ¬(Solid(x) ∧ Liquid(x))"
          
          d) Binary relations:
              - Use two-place predicates for relationships
              - Example: "x likes y" → "Likes(x,y)"
              - Example: "x is part of y" → "Part(x,y)"

        5. Quantification Guidelines:
          - Use ∀x only for universal claims about categories
          - Don't quantify simple implications
          - For relationships, specify both entities
          Example: "Every part of machine M works" → "∀x (Part(x,m) → Works(x))"       

        6. Statement structure:
          - Universal statements should use appropriate quantification
          - Existential claims should use existential quantifiers
          - Simple implications should not be unnecessarily quantified
          - Individual claims should use constants
          - Negations should be properly scoped

        7. Negation and Antonym Handling:
          - For explicit negations ("not"): use ¬ operator
            Example: "not mortal" → "¬Mortal(x)"
          
          - For antonyms and implicit negations:
            * "immortal" → "¬Mortal(x)"
            * "infinite" → "¬Finite(x)"
            * "impossible" → "¬Possible(x)"
          
          - Never create new predicates for negated concepts
            * WRONG: "Immortal(x)"
            * RIGHT: "¬Mortal(x)"

        8. Common Patterns with Negation:
          - "A is immortal" → "¬Mortal(a)"
          - "No A is B" → "∀x (A(x) → ¬B(x))"
          - "A is not B" → "¬B(a)"
          - "Some A are not B" → "∃x (A(x) ∧ ¬B(x))"

        9. Category and Property Rules:
          - "All A are B" must use "∀x (A(x) → B(x))"
          - "[Group] are [Category]" must use "∀x (Group(x) → Category(x))"
          - Individual properties use direct predication: "P(a)"
          - Negated properties use "¬P(a)" not "P(¬a)"

        10. Quantification Guidelines:
          - Use ∀x for universal claims about categories
          - Use ∃x only for explicit existential claims
          - Don't quantify simple cause-effect relationships
          - Maintain quantification in transitive reasoning

        11. Consistency Requirements:
          - Keep predicate arity consistent throughout
          - Use same constant names for same entities
          - Preserve quantification in related statements
          - Match conclusion form to premise structure

    CRITICAL: Check the exact meaning of the conclusion:
    - Read the original conclusion carefully
    - Preserve negations exactly as they appear
    - Don't change the meaning of the conclusion
    
    Examples of conclusion handling:
    Original → Logical Form
    "is mortal" → "Mortal(x)"
    "is immortal" → "¬Mortal(x)"
    "is not mortal" → "¬Mortal(x)"
    "does not die" → "¬Dies(x)"

    Convert the text maintaining logical precision while avoiding unnecessary complexity.""")
    
    # 첫 번째 변환
    messages = [
        prompt, 
        HumanMessage(content=f"Convert this text to logical statements: {state['input']}")
    ]
    
    response = llm.invoke(messages)
    
    # 결과 정리 - 실제 논리식만 추출
    statements = []
    for line in response.content.strip().split("\n"):
        line = line.strip()
        # 실제 논리식만 선택 (번호가 있거나 ∴로 시작하는 줄)
        if line and (line.startswith(('1.', '2.', '3.', '∴')) or '∴' in line):
            # 분석 텍스트 제거
            if not any(x in line.lower() for x in ['verification', 'analysis', 'correction', 'premises:', 'conclusion:']):
                # 불필요한 기호 제거
                line = line.rstrip('.')
                line = line.replace('\\(', '').replace('\\)', '')
                line = line.replace('*', '')
                # 논리식 정리
                if '∴' in line:
                    line = line.split('∴')[-1].strip()
                    line = '∴ ' + line
                elif not line.startswith(('1.', '2.', '3.')):
                    line = f"{len(statements)+1}. {line.lstrip('0123456789. ')}"
                statements.append(line)
    
    # 결론이 없는 경우 처리
    if not any('∴' in s for s in statements):
        conclusion_prompt = HumanMessage(content=f"""
        The original text has this conclusion: "{state['input'].split('Therefore,')[1].strip()}"
        Convert ONLY this conclusion to a logical statement starting with ∴
        Use the same predicates and constants as the premises: {statements}""")
        
        conclusion_response = llm.invoke([prompt, conclusion_prompt])
        conclusion = conclusion_response.content.strip()
        if '∴' in conclusion:
            statements.append(conclusion.split('\n')[0].strip())
    
    state["logical_statements"] = statements
    state["status"] = "parsed"
    state["messages"].append(response)
    state["reasoning"].append("Converted natural language to logical statements")
    
    return state

class LogicalStatement:
    def __init__(self, text: str):
        self.original = text
        self.predicates = set()
        self.constants = set()
        self.variables = set()
        self.quantifiers = set()
        
    def normalize(self) -> str:
        """Normalize the logical statement"""
        text = self.original
        
        # Extract and normalize components
        for match in re.finditer(r'[A-Z][a-zA-Z]*\([^)]+\)', text):
            pred = match.group()
            self.predicates.add(pred.split('(')[0])
            
            # Extract constants (capitalized words inside parentheses)
            for const in re.finditer(r'[A-Z][a-z]+', pred.split('(')[1]):
                self.constants.add(const.group())
                
            # Extract variables
            for var in re.finditer(r'[a-z](?!\w)', pred.split('(')[1]):
                self.variables.add(var.group())
                
        # Extract quantifiers
        self.quantifiers = set(re.findall(r'[∀∃]', text))
        
        # Normalize constants to lowercase
        for const in self.constants:
            text = text.replace(const, const.lower())
            
        return text

class LogicalTransformer:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
    
    def analyze_statement(self, text: str) -> dict:
        """Analyze a single statement and return its logical structure"""
        analyze_prompt = SystemMessage(content="""
        Analyze this statement and return a JSON object with:
        1. The type of statement (premise/conclusion)
        2. The logical meaning (positive/negative)
        3. The predicates involved
        
        Example:
        Input: "Socrates is immortal"
        Output: {
            "type": "conclusion",
            "meaning": "negative",
            "target_predicate": "Mortal",
            "logical_form": "¬Mortal(socrates)"
        }""")
        
        response = self.llm.invoke([analyze_prompt, HumanMessage(content=text)])
        return json.loads(response.content)
    
    def transform(self, text: str) -> list[str]:
        # Split into individual statements
        parts = text.split('Therefore,')
        premises = [p.strip().rstrip('.') for p in parts[0].split('.') if p.strip()]
        conclusion = parts[1].strip().rstrip('.')
        
        statements = []
        
        # Process premises
        for i, premise in enumerate(premises, 1):
            analysis = self.analyze_statement(premise)
            logical_form = analysis.get('logical_form', '')
            if logical_form:
                statements.append(f"{i}. {logical_form}")
        
        # Process conclusion
        conclusion_analysis = self.analyze_statement(conclusion)
        if conclusion_analysis.get('logical_form'):
            statements.append(f"∴ {conclusion_analysis['logical_form']}")
        
        return statements

def natural_to_logical(state: LogicalState) -> LogicalState: # working
    """Convert natural language to logical statements"""
    llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
    
    prompt = SystemMessage(content="""
    Convert the input text to exactly three logical statements.
    
    YOU MUST:
    1. Output EXACTLY these three lines:
       - First line must start with "1. "
       - Second line must start with "2. "
       - Third line must start with "∴ "
    2. Use standard logical notation (∀, ∃, →, ∧, ∨, ¬)
    3. Convert negative meanings (like "immortal") to explicit negations (¬Mortal)
    4. Use uppercase for predicates and lowercase for constants
    
    Example Input: "All A are B. X is A. Therefore, X is not B."
    Example Output:
    1. ∀x (A(x) → B(x))
    2. A(x)
    ∴ ¬B(x)
    
    Output only these three lines, nothing else.""")
    
    try:
        response = llm.invoke([prompt, HumanMessage(content=state['input'])])
        
        # Split response into lines and clean
        lines = [line.strip() for line in response.content.split('\n') if line.strip()]
        
        # Validate exactly three lines
        if len(lines) != 3:
            raise ValueError("Must have exactly three lines")
            
        # Validate line prefixes
        if not (lines[0].startswith('1.') and 
                lines[1].startswith('2.') and 
                lines[2].startswith('∴')):
            raise ValueError("Invalid line prefixes")
        
        # Convert any uppercase constants to lowercase
        lines = [re.sub(r'([A-Z][a-z]+)\)', lambda m: m.group(1).lower() + ')', line) for line in lines]
        
        state["logical_statements"] = lines
        state["status"] = "parsed"
        state["error"] = None
        
    except Exception as e:
        state["logical_statements"] = []
        state["status"] = "error"
        state["error"] = str(e)
    
    state["reasoning"] = ["Converted natural language to logical statements"]
    return state

def natural_to_logical(state: LogicalState) -> LogicalState: # working
    """Convert natural language to logical statements"""
    llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
    
    semantic_prompt = SystemMessage(content="""
    #Role
    - You are world class language and logic expert who is able to invent new logical systems.
    #Purpose
    - Take a deep breath and think step by step. 
    - Convert this natural language argument to predicate logic statements.
    
    **That statements will be used to create a symbolic representation using Z3 in python.**
    
    #Constraints
    YOU MUST:
    0. identify words that express negation(example: antonyms) of words that appear in the previous arguments and modify them to previously appeared words' logical negation.
    
    1. Output EXACTLY three lines in this format:
       1. <first_premise>
       2. <second_premise>
       ∴ <conclusion>
    
    2. Use standard logical notation:
       - Universal quantifier: ∀x
       - Existential quantifier: ∃x
       - Implication: →
       - Negation: ¬
       - Conjunction: ∧
       - Disjunction: ∨
       - Parentheses required for implications: (P → Q)
    
    3. Predicate Format:
       - Single word predicates only (Rain, Wet, Bird)
       - No compound predicates (NOT CompletelySolid, use Solid instead)
       - Example: "completely solid" → "Solid"
    
    4. Constants and Variables:
       - Constants: Single lowercase letter (s, a, t)
       - Variables: x for universal, y for existential
       - Example: "this animal" → "a"
       - Example: "Socrates" → "s"
       - Example: "the machine" → "m"
    
    5. Common Patterns:
       - "No X are Y" → "∀x (X(x) → ¬Y(x))"
       - "All X are Y" → "∀x (X(x) → Y(x))"
       - "Some X are Y" → "∃x (X(x) ∧ Y(x))"
       - "Some X are not Y" → "∃x (X(x) ∧ ¬Y(x))"
       - "Only X have Y" → "∀x (Y(x) → X(x))"
       - "If X then Y" → "(X → Y)" or "∀x (X(x) → Y(x))"
    
    6. Binary Relations:
       - Use parentheses: Likes(x,y)
       - Keep variables consistent
       - Example: "x likes y" → "Likes(x,y)"

    7. Handle negations:
       - Convert negative meanings to ¬
       - Example: "immortal" → "¬Mortal"
       - Example: "not possible" → "¬Possible"
    
    8. Use proper case:
       - Predicates: Uppercase first letter (Human, Mortal)
       - Constants: All lowercase (socrates, plato)
       - Variables: Single lowercase letters (x, y)
    
    ALWAYS verify your output follows above rules exactly.
    """)
    
    try:
        # Get initial response
        response = llm.invoke([semantic_prompt, HumanMessage(content=state['input'])])
        
        # Clean and validate lines
        lines = [line.strip() for line in response.content.split('\n') if line.strip()]
        
        # Strict validation
        if len(lines) != 3:
            raise ValueError(f"Expected 3 lines, got {len(lines)}")
        
        if not lines[0].startswith('1.'):
            raise ValueError("First line must start with '1.'")
        if not lines[1].startswith('2.'):
            raise ValueError("Second line must start with '2.'")
        if not lines[2].startswith('∴'):
            raise ValueError("Third line must start with '∴'")
        
        # Convert any uppercase constants to lowercase
        lines = [re.sub(r'([A-Z][a-z]+)\)', lambda m: m.group(1).lower() + ')', line) for line in lines]
        
        # Additional validation for logical symbols
        required_symbols = ['∀', '→', '¬']
        if not any(symbol in ''.join(lines) for symbol in required_symbols):
            raise ValueError("Missing required logical symbols")
        
        state["logical_statements"] = lines
        state["status"] = "parsed"
        state["error"] = None
        
    except Exception as e:
        state["logical_statements"] = []
        state["status"] = "error"
        state["error"] = str(e)
    
    state["reasoning"] = ["Converted natural language to logical statements"]
    return state

def create_symbolic_representation(state: LogicalState) -> LogicalState:
    """Create Z3 symbolic representation"""
    try:
        # Extract predicates and constants
        predicates = extract_predicates(state["logical_statements"])
        constants = extract_constants(state["logical_statements"])
        
        # Create solver
        solver = Solver()
        
        # Create sorts and store predicates
        state["predicates"] = {}
        for pred in predicates:
            state["predicates"][pred] = Function(pred, IntSort(), BoolSort())
            
        # Create and store constants
        state["constants"] = {}
        for const in constants:
            state["constants"][const] = Int(const)
            
        # Process each logical statement
        for statement in state["logical_statements"]:
            if "∀" in statement:
                # Handle universal quantification
                x = Int('x')
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', statement)
                if len(pred_matches) >= 2:
                    antecedent = state["predicates"][pred_matches[0]](x)
                    consequent = state["predicates"][pred_matches[1]](x)
                    solver.add(ForAll([x], Implies(antecedent, consequent)))
                    
            elif "∃" in statement:
                # Handle existential quantification
                x = Int('x')
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', statement)
                if pred_matches:
                    conditions = []
                    for pred in pred_matches:
                        conditions.append(state["predicates"][pred](x))
                    solver.add(Exists([x], And(conditions)))
                    
            elif "→" in statement:
                # Handle direct implications
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(([a-z]+)\)', statement)
                if len(pred_matches) >= 2:
                    const = state["constants"][pred_matches[0][1]]
                    antecedent = state["predicates"][pred_matches[0][0]](const)
                    consequent = state["predicates"][pred_matches[1][0]](const)
                    solver.add(Implies(antecedent, consequent))
                    
            elif "∴" not in statement:
                # Handle basic predicates
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(([a-z]+)\)', statement)
                for pred, const in pred_matches:
                    if "¬" in statement:
                        solver.add(Not(state["predicates"][pred](state["constants"][const])))
                    else:
                        solver.add(state["predicates"][pred](state["constants"][const]))
        
        state["symbolic_representation"] = solver
        state["status"] = "symbolized"
        state["reasoning"].append("Created Z3 symbolic representation")
        
    except Exception as e:
        state["status"] = "error"
        state["error"] = str(e)
        state["reasoning"].append(f"Error in symbolic representation: {str(e)}")
    
    return state

def evaluate_logic(state: LogicalState) -> LogicalState:
    """Evaluate logical statements using Z3"""
    try:
        if not isinstance(state["symbolic_representation"], Solver):
            raise ValueError("Invalid symbolic representation")
            
        # Create a new solver for checking the conclusion
        proof_solver = Solver()
        
        # Copy all premises
        for assertion in state["symbolic_representation"].assertions():
            proof_solver.add(assertion)
            
        # Find conclusion and parse its structure
        conclusion = next((stmt for stmt in state["logical_statements"] if "∴" in stmt), None)
        if not conclusion:
            raise ValueError("No conclusion found in statements")
            
        # Clean up conclusion for parsing
        clean_conclusion = conclusion.replace("∴", "").strip()
        clean_conclusion = re.sub(r'^\d+\.\s*', '', clean_conclusion)
        
        # Parse conclusion structure
        if "∀" in clean_conclusion:
            # Handle universal conclusion
            x = Int('x')
            pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', clean_conclusion)
            if len(pred_matches) >= 2:
                antecedent = state["predicates"][pred_matches[0]](x)
                consequent = state["predicates"][pred_matches[1]](x)
                conclusion_expr = ForAll([x], Implies(antecedent, consequent))
        elif "∃" in clean_conclusion:
            # Handle existential conclusion
            x = Int('x')
            pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', clean_conclusion)
            conditions = []
            for pred in pred_matches:
                if "¬" in clean_conclusion.split(pred)[0].strip():
                    conditions.append(Not(state["predicates"][pred](x)))
                else:
                    conditions.append(state["predicates"][pred](x))
            conclusion_expr = Exists([x], And(conditions) if len(conditions) > 1 else conditions[0])
        else:
            # Handle simple predicate conclusion
            pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(([a-z]+)\)', clean_conclusion)
            if not pred_matches:
                raise ValueError("Could not parse conclusion")
                
            pred, const = pred_matches[0]
            pred_func = state["predicates"][pred]
            const_val = state["constants"][const]
            
            # Handle negation in conclusion
            if "¬" in clean_conclusion:
                conclusion_expr = Not(pred_func(const_val))
            else:
                conclusion_expr = pred_func(const_val)
        
        # Create check solver for validity
        check_solver = Solver()
        for assertion in proof_solver.assertions():
            check_solver.add(assertion)
            
        # Add negation of conclusion to check validity
        check_solver.add(Not(conclusion_expr))
        
        result = check_solver.check()
        
        if result == unsat:
            state["result"] = True
            state["status"] = "valid"
            state["reasoning"].append(
                "The conclusion logically follows from the premises"
            )
        else:
            # Check for contradiction
            contra_solver = Solver()
            for assertion in proof_solver.assertions():
                contra_solver.add(assertion)
            contra_solver.add(conclusion_expr)
            
            if contra_solver.check() == unsat:
                state["result"] = False
                state["status"] = "contradiction"
                state["reasoning"].append(
                    "The premises and conclusion form a contradiction"
                )
            else:
                state["result"] = False
                state["status"] = "invalid"
                state["reasoning"].extend([
                    "The conclusion does not logically follow from the premises",
                    "A counterexample exists where premises are true but conclusion is false"
                ])
                
                # Get model for counterexample if available
                if result == sat:
                    model = check_solver.model()
                    state["reasoning"].append(
                        "Counterexample found in the model"
                    )
                
    except Exception as e:
        state["status"] = "error"
        state["error"] = str(e)
        state["reasoning"].append(f"Error in evaluation: {str(e)}")
    
    return state

def create_logical_agent() -> StateGraph:
    """Create the logical reasoning agent workflow"""
    workflow = StateGraph(LogicalState)
    
    workflow.add_node("parse", natural_to_logical)
    workflow.add_node("symbolize", create_symbolic_representation)
    workflow.add_node("evaluate", evaluate_logic)
    
    workflow.add_edge("parse", "symbolize")
    workflow.add_edge("symbolize", "evaluate")
    
    workflow.set_entry_point("parse")
    
    return workflow.compile()

def run_logical_agent(input_text: str, context: Optional[Dict] = None) -> Dict:
    """Run the logical reasoning agent"""
    agent = create_logical_agent()
    
    initial_state = LogicalState(
        input=input_text,
        logical_statements=[],
        symbolic_representation=None,
        status="starting",
        error=None,
        result=None,
        context=context or {},
        messages=[],
        reasoning=[]
    )
    
    final_state = agent.invoke(initial_state)
    return {
        "input": final_state["input"],
        "logical_statements": final_state["logical_statements"],
        "status": final_state["status"],
        "error": final_state["error"],
        "result": final_state["result"],
        "reasoning": final_state["reasoning"]
    }

if __name__ == "__main__":
    # Test the system
    test_input = "All humans are mortal. Socrates is human. Therefore, Socrates is immortal."
    result = run_logical_agent(test_input)
    print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "input": "All humans are mortal. Socrates is human. Therefore, Socrates is immortal.",
  "logical_statements": [
    "1. ∀x (Human(x) → Mortal(x))",
    "2. Human(s)",
    "∴ ¬Mortal(s)"
  ],
  "status": "contradiction",
  "error": null,
  "result": false,
  "reasoning": [
    "Converted natural language to logical statements",
    "Created Z3 symbolic representation",
    "The premises and conclusion form a contradiction"
  ]
}


In [137]:
# Test cases
tests = [
    "All humans are mortal. Socrates is human. Therefore, Socrates is immortal.",
    "If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.",
    "Every student studies. Some people don't study. Therefore, some people are not students.",
    "All birds fly. Penguins are birds. Therefore, penguins fly."
]

for test in tests:
    result = run_logical_agent(test)
    print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "input": "All humans are mortal. Socrates is human. Therefore, Socrates is immortal.",
  "logical_statements": [
    "1. ∀x (Human(x) → Mortal(x))",
    "2. Human(s)",
    "∴ ¬Mortal(s)"
  ],
  "status": "contradiction",
  "error": null,
  "result": false,
  "reasoning": [
    "Converted natural language to logical statements",
    "Created Z3 symbolic representation",
    "The premises and conclusion form a contradiction"
  ]
}
{
  "input": "If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.",
  "logical_statements": [
    "1. ∀x (Rain(x) → Wet(x))",
    "2. ¬Wet(g)",
    "∴ ¬Rain(g)"
  ],
  "status": "valid",
  "error": null,
  "result": true,
  "reasoning": [
    "Converted natural language to logical statements",
    "Created Z3 symbolic representation",
    "The conclusion logically follows from the premises"
  ]
}
{
  "input": "Every student studies. Some people don't study. Therefore, some people are not students.",
  "logical_statements": [
 

In [138]:
# Test cases
tests = [
    "If something is not impossible, then it is possible. This is not impossible. Therefore, this is possible.",
    "All mathematicians are logical. Some logical people are not creative. Therefore, some mathematicians are not creative.",
    "Only mammals have fur. This animal has no fur. Therefore, this animal is not a mammal.",
    "No reptiles are mammals. All snakes are reptiles. Therefore, no snakes are mammals.",
    "Everyone has someone who likes them. Alice likes nobody. Therefore, Alice is liked by someone.",
    "Every part of this machine is working. If every part is working, then the machine is functional. Therefore, this machine is functional.",
    "Nothing can be both completely solid and completely liquid at the same time. This substance is completely solid. Therefore, this substance is not completely liquid."
]

for test in tests:
    result = run_logical_agent(test)
    print(json.dumps(result, indent=2, ensure_ascii=False))
    print("\n" + "="*80 + "\n")

{
  "input": "If something is not impossible, then it is possible. This is not impossible. Therefore, this is possible.",
  "logical_statements": [
    "1. ∀x (¬Impossible(x) → Possible(x))",
    "2. ¬Impossible(a)",
    "∴ Possible(a)"
  ],
  "status": "invalid",
  "error": null,
  "result": false,
  "reasoning": [
    "Converted natural language to logical statements",
    "Created Z3 symbolic representation",
    "The conclusion does not logically follow from the premises",
    "A counterexample exists where premises are true but conclusion is false",
    "Counterexample found in the model"
  ]
}


{
  "input": "All mathematicians are logical. Some logical people are not creative. Therefore, some mathematicians are not creative.",
  "logical_statements": [
    "1. ∀x (Mathematician(x) → Logical(x))",
    "2. ∃x (Logical(x) ∧ ¬Creative(x))",
    "∴ ∃x (Mathematician(x) ∧ ¬Creative(x))"
  ],
  "status": "invalid",
  "error": null,
  "result": false,
  "reasoning": [
    "Converted nat

In [187]:
from typing import TypedDict, Dict, Any, List, Optional
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph
import json
from z3 import *
import re

class LogicalState(TypedDict):
    input: str
    logical_statements: List[str]
    predicates: Dict[str, Any]  # Store predicate functions
    constants: Dict[str, Any]   # Store constants
    symbolic_representation: Any
    status: str
    error: Optional[str]
    result: Optional[bool]
    context: Dict[str, Any]
    messages: List[BaseMessage]
    reasoning: List[str]

def extract_predicates(statements: List[str]) -> List[str]:
    """Extract predicate names from logical statements"""
    predicates = set()
    pattern = r'([A-Z][a-zA-Z]*)\('
    
    for stmt in statements:
        matches = re.findall(pattern, stmt)
        predicates.update(matches)
    
    return list(predicates)

def extract_constants(statements: List[str]) -> List[str]:
    """Extract constant names from logical statements"""
    constants = set()
    pattern = r'\(([a-z][a-zA-Z]*)\)'
    
    for stmt in statements:
        matches = re.findall(pattern, stmt)
        constants.update(matches)
    
    return list(constants)

def natural_to_logical(state: LogicalState) -> LogicalState: # working
    """Convert natural language to logical statements"""
    llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
    
    semantic_prompt = SystemMessage(content="""
    #Role
    - You are world class language and logic expert who is able to invent new logical systems.
    #Purpose
    - Take a deep breath and think step by step. 
    - Convert this natural language argument to predicate logic statements.
    
    **That statements will be used to create a symbolic representation using Z3 in python.**
    
    #Constraints
    YOU MUST:
    0. identify words that express negation(example: antonyms) of words that appear in the previous arguments and modify them to previously appeared words' logical negation.
    
    1. Output EXACTLY three lines in this format:
       1. <first_premise>
       2. <second_premise>
       ∴ <conclusion>
    
    2. Use standard logical notation:
       - Universal quantifier: ∀x
       - Existential quantifier: ∃x
       - Implication: →
       - Negation: ¬
       - Conjunction: ∧
       - Disjunction: ∨
       - Parentheses required for implications: (P → Q)
    
    3. Predicate Format:
       - Single word predicates only (Rain, Wet, Bird)
       - No compound predicates (NOT CompletelySolid, use Solid instead)
       - Example: "completely solid" → "Solid"
    
    4. Constants and Variables:
       - Constants: Single lowercase letter (s, a, t)
       - Variables: x for universal, y for existential
       - Example: "this animal" → "a"
       - Example: "Socrates" → "s"
       - Example: "the machine" → "m"
    
    5. Common Patterns:
       - "No X are Y" → "∀x (X(x) → ¬Y(x))"
       - "All X are Y" → "∀x (X(x) → Y(x))"
       - "Some X are Y" → "∃x (X(x) ∧ Y(x))"
       - "Some X are not Y" → "∃x (X(x) ∧ ¬Y(x))"
       - "Only X have Y" → "∀x (Y(x) → X(x))"
       - "If X then Y" → "(X → Y)" or "∀x (X(x) → Y(x))"
    
    6. Binary Relations:
       - Use parentheses: Likes(x,y)
       - Keep variables consistent
       - Example: "x likes y" → "Likes(x,y)"

    7. Handle negations:
       - Convert negative meanings to ¬
       - Example: "immortal" → "¬Mortal"
       - Example: "not possible" → "¬Possible"
    
    8. Use proper case:
       - Predicates: Uppercase first letter (Human, Mortal)
       - Constants: All lowercase (socrates, plato)
       - Variables: Single lowercase letters (x, y)
    
    ALWAYS verify your output follows above rules exactly.
    """)
    
    try:
        # Get initial response
        response = llm.invoke([semantic_prompt, HumanMessage(content=state['input'])])
        
        # Clean and validate lines
        lines = [line.strip() for line in response.content.split('\n') if line.strip()]
        
        # Strict validation
        if len(lines) != 3:
            raise ValueError(f"Expected 3 lines, got {len(lines)}")
        
        if not lines[0].startswith('1.'):
            raise ValueError("First line must start with '1.'")
        if not lines[1].startswith('2.'):
            raise ValueError("Second line must start with '2.'")
        if not lines[2].startswith('∴'):
            raise ValueError("Third line must start with '∴'")
        
        # Convert any uppercase constants to lowercase
        lines = [re.sub(r'([A-Z][a-z]+)\)', lambda m: m.group(1).lower() + ')', line) for line in lines]
        
        # Additional validation for logical symbols
        required_symbols = ['∀', '→', '¬']
        if not any(symbol in ''.join(lines) for symbol in required_symbols):
            raise ValueError("Missing required logical symbols")
        
        state["logical_statements"] = lines
        state["status"] = "parsed"
        state["error"] = None
        
    except Exception as e:
        state["logical_statements"] = []
        state["status"] = "error"
        state["error"] = str(e)
    
    state["reasoning"] = ["Converted natural language to logical statements"]
    return state

def create_symbolic_representation(state: LogicalState) -> LogicalState:
    """Create Z3 symbolic representation"""
    try:
        # Extract predicates and constants
        predicates = extract_predicates(state["logical_statements"])
        constants = extract_constants(state["logical_statements"])
        
        # Create solver
        solver = Solver()
        
        # Create sorts and store predicates
        state["predicates"] = {}
        for pred in predicates:
            state["predicates"][pred] = Function(pred, IntSort(), BoolSort())
            
        # Create and store constants
        state["constants"] = {}
        for const in constants:
            state["constants"][const] = Int(const)
            
        # Process each logical statement
        for statement in state["logical_statements"]:
            if "∀" in statement:
                # Handle universal quantification
                x = Int('x')
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', statement)
                if len(pred_matches) >= 2:
                    antecedent = state["predicates"][pred_matches[0]](x)
                    consequent = state["predicates"][pred_matches[1]](x)
                    solver.add(ForAll([x], Implies(antecedent, consequent)))
                    
            elif "∃" in statement:
                # Handle existential quantification
                x = Int('x')
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', statement)
                if pred_matches:
                    conditions = []
                    for pred in pred_matches:
                        conditions.append(state["predicates"][pred](x))
                    solver.add(Exists([x], And(conditions)))
                    
            elif "→" in statement:
                # Handle direct implications
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(([a-z]+)\)', statement)
                if len(pred_matches) >= 2:
                    const = state["constants"][pred_matches[0][1]]
                    antecedent = state["predicates"][pred_matches[0][0]](const)
                    consequent = state["predicates"][pred_matches[1][0]](const)
                    solver.add(Implies(antecedent, consequent))
                    
            elif "∴" not in statement:
                # Handle basic predicates
                pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(([a-z]+)\)', statement)
                for pred, const in pred_matches:
                    if "¬" in statement:
                        solver.add(Not(state["predicates"][pred](state["constants"][const])))
                    else:
                        solver.add(state["predicates"][pred](state["constants"][const]))
        
        state["symbolic_representation"] = solver
        state["status"] = "symbolized"
        state["reasoning"].append("Created Z3 symbolic representation")
        
    except Exception as e:
        state["status"] = "error"
        state["error"] = str(e)
        state["reasoning"].append(f"Error in symbolic representation: {str(e)}")
    
    return state

def evaluate_logic(state: LogicalState) -> LogicalState:
    """Evaluate logical statements using Z3"""
    try:
        if not isinstance(state["symbolic_representation"], Solver):
            raise ValueError("Invalid symbolic representation")
            
        # Create a new solver for checking the conclusion
        proof_solver = Solver()
        
        # Copy all premises
        for assertion in state["symbolic_representation"].assertions():
            proof_solver.add(assertion)
            
        # Find conclusion and parse its structure
        conclusion = next((stmt for stmt in state["logical_statements"] if "∴" in stmt), None)
        if not conclusion:
            raise ValueError("No conclusion found in statements")
            
        # Clean up conclusion for parsing
        clean_conclusion = conclusion.replace("∴", "").strip()
        clean_conclusion = re.sub(r'^\d+\.\s*', '', clean_conclusion)
        
        # Parse conclusion structure
        if "∀" in clean_conclusion:
            # Handle universal conclusion
            x = Int('x')
            pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', clean_conclusion)
            if len(pred_matches) >= 2:
                antecedent = state["predicates"][pred_matches[0]](x)
                consequent = state["predicates"][pred_matches[1]](x)
                conclusion_expr = ForAll([x], Implies(antecedent, consequent))
        elif "∃" in clean_conclusion:
            # Handle existential conclusion
            x = Int('x')
            pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(x\)', clean_conclusion)
            conditions = []
            for pred in pred_matches:
                if "¬" in clean_conclusion.split(pred)[0].strip():
                    conditions.append(Not(state["predicates"][pred](x)))
                else:
                    conditions.append(state["predicates"][pred](x))
            conclusion_expr = Exists([x], And(conditions) if len(conditions) > 1 else conditions[0])
        else:
            # Handle simple predicate conclusion
            pred_matches = re.findall(r'([A-Z][a-zA-Z]*)\(([a-z]+)\)', clean_conclusion)
            if not pred_matches:
                raise ValueError("Could not parse conclusion")
                
            pred, const = pred_matches[0]
            pred_func = state["predicates"][pred]
            const_val = state["constants"][const]
            
            # Handle negation in conclusion
            if "¬" in clean_conclusion:
                conclusion_expr = Not(pred_func(const_val))
            else:
                conclusion_expr = pred_func(const_val)
        
        # Create check solver for validity
        check_solver = Solver()
        for assertion in proof_solver.assertions():
            check_solver.add(assertion)
            
        # Add negation of conclusion to check validity
        check_solver.add(Not(conclusion_expr))
        
        result = check_solver.check()
        
        if result == unsat:
            state["result"] = True
            state["status"] = "valid"
            state["reasoning"].append(
                "The conclusion logically follows from the premises"
            )
        else:
            # Check for contradiction
            contra_solver = Solver()
            for assertion in proof_solver.assertions():
                contra_solver.add(assertion)
            contra_solver.add(conclusion_expr)
            
            if contra_solver.check() == unsat:
                state["result"] = False
                state["status"] = "contradiction"
                state["reasoning"].append(
                    "The premises and conclusion form a contradiction"
                )
            else:
                state["result"] = False
                state["status"] = "invalid"
                state["reasoning"].extend([
                    "The conclusion does not logically follow from the premises",
                    "A counterexample exists where premises are true but conclusion is false"
                ])
                
                # Get model for counterexample if available
                if result == sat:
                    model = check_solver.model()
                    state["reasoning"].append(
                        "Counterexample found in the model"
                    )
                
    except Exception as e:
        state["status"] = "error"
        state["error"] = str(e)
        state["reasoning"].append(f"Error in evaluation: {str(e)}")
    
    return state

def create_logical_agent() -> StateGraph:
    """Create the logical reasoning agent workflow"""
    workflow = StateGraph(LogicalState)
    
    workflow.add_node("parse", natural_to_logical)
    workflow.add_node("symbolize", create_symbolic_representation)
    workflow.add_node("evaluate", evaluate_logic)
    
    workflow.add_edge("parse", "symbolize")
    workflow.add_edge("symbolize", "evaluate")
    
    workflow.set_entry_point("parse")
    
    return workflow.compile()

def run_logical_agent(input_text: str, context: Optional[Dict] = None) -> Dict:
    """Run the logical reasoning agent"""
    agent = create_logical_agent()
    
    initial_state = LogicalState(
        input=input_text,
        logical_statements=[],
        symbolic_representation=None,
        status="starting",
        error=None,
        result=None,
        context=context or {},
        messages=[],
        reasoning=[]
    )
    
    final_state = agent.invoke(initial_state)
    return {
        "input": final_state["input"],
        "logical_statements": final_state["logical_statements"],
        "status": final_state["status"],
        "error": final_state["error"],
        "result": final_state["result"],
        "reasoning": final_state["reasoning"]
    }

if __name__ == "__main__":
    # Test the system
    test_input = "All humans are mortal. Socrates is human. Therefore, Socrates is immortal."
    result = run_logical_agent(test_input)
    print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "input": "All humans are mortal. Socrates is human. Therefore, Socrates is immortal.",
  "logical_statements": [
    "1. ∀x (Human(x) → Mortal(x))",
    "2. Human(s)",
    "∴ ¬Mortal(s)"
  ],
  "status": "contradiction",
  "error": null,
  "result": false,
  "reasoning": [
    "Converted natural language to logical statements",
    "Created Z3 symbolic representation",
    "The premises and conclusion form a contradiction"
  ]
}


In [198]:
from typing import Literal, TypedDict, List, Optional, Dict, Any, Annotated
from langgraph.types import Command
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dataclasses import dataclass
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import re
from z3 import *

class LogicalState(TypedDict):
    """논리 추론 상태"""
    input: str
    messages: Annotated[List[HumanMessage | AIMessage], add_messages]  # 메시지 히스토리
    logical_statements: List[str]
    status: str
    error: Optional[str]
    result: Optional[bool]
    reasoning: List[str]

def create_z3_predicates(predicate_name: str) -> FuncDeclRef:
    """Z3 predicate 생성"""
    return Function(predicate_name, IntSort(), BoolSort())

def parse_predicate(expr: str) -> tuple:
    """술어 논리 파싱"""
    match = re.match(r'([A-Z][a-z]*)\(([a-z])\)', expr)
    if match:
        return match.group(1), match.group(2)
    return None, None

def parse_quantifier(expr: str) -> tuple:
    """양화사 파싱"""
    match = re.match(r'[∀∃]([a-z])\s*\((.*)\)', expr)
    if match:
        return match.group(1), match.group(2)
    return None, None

def convert_to_z3(stmt: str, predicates: Dict[str, FuncDeclRef]) -> Any:
    """논리식을 Z3 formula로 변환"""
    x = Int('x')
    
    # 양화사 처리
    if stmt.startswith('∀'):
        var, rest = parse_quantifier(stmt)
        if var:
            x = Int(var)
            return ForAll([x], convert_to_z3(rest, predicates))
    elif stmt.startswith('∃'):
        var, rest = parse_quantifier(stmt)
        if var:
            x = Int(var)
            return Exists([x], convert_to_z3(rest, predicates))
            
    # 논리 연산자 처리
    if '→' in stmt:
        left, right = stmt.split('→')
        return Implies(convert_to_z3(left.strip(), predicates),
                     convert_to_z3(right.strip(), predicates))
    elif '∧' in stmt:
        left, right = stmt.split('∧')
        return And(convert_to_z3(left.strip(), predicates),
                  convert_to_z3(right.strip(), predicates))
    elif '¬' in stmt:
        return Not(convert_to_z3(stmt[1:].strip(), predicates))
        
    # 기본 술어 처리
    pred_name, var = parse_predicate(stmt)
    if pred_name and var in predicates:
        return predicates[pred_name](Int(var))
        
    return None

In [203]:
def parse_natural_language(state: LogicalState) -> Dict:
    """자연어를 논리 문장으로 변환"""
    input_text = state["input"]
    statements = []
    reasoning = ["Converted natural language to logical statements"]
    
    # 문장들을 분리
    sentences = [s.strip() for s in input_text.split('.') if s.strip()]
    if len(sentences) < 2:
        return {
            "status": "error",
            "error": "Not enough statements",
            "reasoning": reasoning + ["Not enough statements to form a logical argument"]
        }
    
    # 전제들 처리
    premises = sentences[:-1]
    for i, premise in enumerate(premises, 1):
        # "All A are B" 패턴
        if premise.startswith(("All", "Every")):
            match = re.match(r"(?:All|Every)\s+(\w+)s?\s+(?:are|is)\s+(\w+)", premise)
            if match:
                subject, predicate = match.groups()
                # 첫 글자를 대문자로 변환
                subject = subject.capitalize()
                predicate = predicate.capitalize()
                statements.append(f"{i}. ∀x ({subject}(x) → {predicate}(x))")
                continue
        
        # "If A then B" 패턴
        if premise.startswith("If"):
            match = re.match(r"If\s+it\s+(\w+)s?,\s+the\s+(\w+)\s+is\s+(\w+)", premise)
            if match:
                cond, subj, pred = match.groups()
                # 첫 글자를 대문자로 변환
                cond = cond.capitalize()
                pred = pred.capitalize()
                statements.append(f"{i}. {cond}(x) → {pred}(x)")
                continue
        
        # "X is not Y" 패턴
        if "not" in premise:
            match = re.match(r"(?:The\s+)?(\w+)\s+is\s+not\s+(\w+)", premise)
            if match:
                subject, predicate = match.groups()
                # 첫 글자를 대문자로 변환
                predicate = predicate.capitalize()
                statements.append(f"{i}. ¬{predicate}({subject.lower()[0]})")
                continue
        
        # "X is Y" 패턴
        match = re.match(r"(\w+)\s+(?:is|are)\s+(\w+)s?", premise)
        if match:
            subject, predicate = match.groups()
            # 첫 글자를 대문자로 변환
            predicate = predicate.capitalize()
            statements.append(f"{i}. {predicate}({subject.lower()[0]})")
    
    # 결론 처리
    conclusion = sentences[-1]
    if "Therefore" in conclusion:
        conclusion = conclusion.split("Therefore,")[1].strip()
    
    # "X is not Y" 패턴
    if "not" in conclusion:
        match = re.match(r"(?:the\s+)?(\w+)\s+(?:is|did|are)\s+not\s+(\w+)", conclusion)
        if match:
            subject, predicate = match.groups()
            # 첫 글자를 대문자로 변환
            predicate = predicate.capitalize()
            statements.append(f"∴ ¬{predicate}({subject.lower()[0]})")
    else:
        # "X is Y" 패턴
        match = re.match(r"(\w+)\s+(?:is|are)\s+(\w+)", conclusion)
        if match:
            subject, predicate = match.groups()
            # 첫 글자를 대문자로 변환
            predicate = predicate.capitalize()
            statements.append(f"∴ {predicate}({subject.lower()[0]})")
    
    if not statements:
        return {
            "status": "error",
            "error": "Failed to parse statements",
            "reasoning": reasoning + ["Failed to parse logical statements"]
        }
    
    return {
        "logical_statements": statements,
        "status": "parsed",
        "reasoning": reasoning
    }

def evaluate_with_z3(state: LogicalState) -> Dict:
    """Z3를 사용하여 논리적 타당성 평가"""
    statements = state.get("logical_statements", [])
    if not statements:
        return {
            "status": "error",
            "error": "No statements to evaluate",
            "reasoning": ["No logical statements found"]
        }
    
    try:
        # 결론과 전제 분리
        premises = []
        conclusion = None
        
        for stmt in statements:
            if stmt.startswith('∴'):
                conclusion = stmt[2:].strip()
            else:
                premises.append(stmt[3:].strip())  # "1. " 제거
        
        if not conclusion:
            return {
                "status": "error",
                "error": "No conclusion found",
                "reasoning": ["No conclusion found in statements"]
            }
        
        # Z3 solver 생성
        solver = Solver()
        
        # 도메인 선언
        Universe = DeclareSort('Universe')
        
        # 술어와 상수 수집
        predicates = {}
        constants = {}
        
        def collect_symbols(formula: str):
            # 술어 수집 (대문자로 시작하는 이름)
            for match in re.finditer(r'([A-Z][a-zA-Z]*)\(', formula):
                pred_name = match.group(1)
                if pred_name not in predicates:
                    predicates[pred_name] = Function(pred_name, Universe, BoolSort())
            
            # 상수 수집 (소문자)
            for match in re.finditer(r'\(([a-z])\)', formula):
                const_name = match.group(1)
                if const_name not in constants:
                    constants[const_name] = Const(const_name, Universe)
        
        # 모든 문장에서 술어와 상수 수집
        for stmt in statements:
            collect_symbols(stmt)
        
        def parse_formula(formula: str) -> ExprRef:
            """논리식을 Z3 formula로 변환"""
            formula = formula.strip()
            
            # 전칭 문장 (∀x)
            if formula.startswith('∀x'):
                match = re.match(r'∀x \((\w+)\(x\) → (\w+)\(x\)\)', formula)
                if match:
                    P, Q = match.groups()
                    x = Const('x', Universe)
                    return ForAll([x], Implies(predicates[P](x), predicates[Q](x)))
            
            # 부정 (¬)
            if formula.startswith('¬'):
                inner = formula[1:].strip()
                match = re.match(r'(\w+)\(([a-z])\)', inner)
                if match:
                    pred_name, const_name = match.groups()
                    return Not(predicates[pred_name](constants[const_name]))
            
            # 조건문 (→)
            if '→' in formula:
                left, right = [s.strip() for s in formula.split('→')]
                return Implies(parse_formula(left), parse_formula(right))
            
            # 단순 술어
            match = re.match(r'(\w+)\(([a-z])\)', formula)
            if match:
                pred_name, const_name = match.groups()
                return predicates[pred_name](constants[const_name])
            
            raise ValueError(f"Unsupported formula: {formula}")

        # 결론의 부정을 추가하여 타당성 검사
        try:
            solver.push()
            negated_conclusion = Not(parse_formula(conclusion))
            solver.add(negated_conclusion)
            result = solver.check()
            solver.pop()
            
            if result == unsat:
                return {
                    "status": "valid",
                    "result": True,
                    "reasoning": [
                        "The conclusion logically follows from the premises"
                    ]
                }
            elif result == sat:
                model = solver.model()
                return {
                    "status": "invalid",
                    "result": False,
                    "reasoning": [
                        "The conclusion does not logically follow from the premises",
                        "A counterexample exists where premises are true but conclusion is false"
                    ]
                }
            else:
                return {
                    "status": "contradiction",
                    "result": False,
                    "reasoning": [
                        "The premises and conclusion form a contradiction"
                    ]
                }
        except Exception as e:
                    return {
                        "status": "error",
                        "error": f"Error evaluating conclusion: {str(e)}",
                        "reasoning": [f"Failed to evaluate conclusion: {conclusion}"]
                    }
                    
    except Exception as e:
                return {
                    "status": "error",
                    "error": str(e),
                    "reasoning": [f"Error in evaluation: {str(e)}"]
                }

def create_logical_reasoning_chain():
    """논리 추론 체인 생성"""
    workflow = StateGraph(LogicalState)
    
    def parse_step(state: LogicalState) -> Dict:
        """파싱 단계"""
        result = parse_natural_language(state)
        return {
            **result,
            "messages": [
                HumanMessage(content=state["input"]),
                AIMessage(content=f"Parsed statements:\n" + "\n".join(result.get("logical_statements", [])))
            ]
        }
    
    def evaluate_step(state: LogicalState) -> Dict:
        """평가 단계"""
        result = evaluate_with_z3(state)
        return {
            **result,
            "messages": [
                AIMessage(content=f"Evaluation result: {result['status']}\n" + "\n".join(result["reasoning"]))
            ]
        }
    
    # 노드 추가
    workflow.add_node("parse", parse_step)
    workflow.add_node("evaluate", evaluate_step)
    
    # 엣지 추가
    workflow.add_edge("parse", "evaluate")
    workflow.add_edge("evaluate", END)
    
    # 시작점 설정
    workflow.set_entry_point("parse")
    
    return workflow.compile()

def process_logical_argument(input_text: str) -> Dict:
    """논리 추론 실행"""
    chain = create_logical_reasoning_chain()
    
    # 초기 상태 생성
    initial_state = {
        "input": input_text,
        "messages": [],
        "logical_statements": [],
        "status": "pending",
        "error": None,
        "result": None,
        "reasoning": []
    }
    
    try:
        return chain.invoke(initial_state)
    except Exception as e:
        return {
            "input": input_text,
            "messages": [AIMessage(content=f"Error: {str(e)}")],
            "logical_statements": [],
            "status": "error",
            "error": str(e),
            "result": None,
            "reasoning": [f"Error in execution: {str(e)}"]
        }

# 테스트
if __name__ == "__main__":
    test_cases = [
        "All humans are mortal. Socrates is human. Therefore, Socrates is immortal.",
        "If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.",
        "All birds fly. Penguins are birds. Therefore, penguins fly."
    ]
    
    for test in test_cases:
        print("\nTesting:", test)
        result = process_logical_argument(test)
        print("Status:", result["status"])
        print("Result:", result["result"])
        print("Reasoning:", result["reasoning"])
        if result["error"]:
            print("Error:", result["error"])


Testing: All humans are mortal. Socrates is human. Therefore, Socrates is immortal.
Status: invalid
Result: False
Reasoning: ['The conclusion does not logically follow from the premises', 'A counterexample exists where premises are true but conclusion is false']

Testing: If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.
Status: invalid
Result: False
Reasoning: ['The conclusion does not logically follow from the premises', 'A counterexample exists where premises are true but conclusion is false']

Testing: All birds fly. Penguins are birds. Therefore, penguins fly.
Status: error
Result: None
Reasoning: ['No conclusion found in statements']
Error: No conclusion found


In [218]:
from typing import TypedDict, List, Optional, Dict, Annotated
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

class LogicalState(TypedDict):
    """논리 추론 상태"""
    input: str
    messages: Annotated[List[HumanMessage | AIMessage], add_messages]
    status: str
    result: Optional[bool]
    reasoning: List[str]

def evaluate_logic(state: LogicalState) -> Dict:
    """논리적 타당성 평가"""
    input_text = state["input"]
    
    try:
        # 문장 분리
        sentences = [s.strip() for s in input_text.split('.') if s.strip()]
        if len(sentences) < 2:
            return {
                "status": "error",
                "result": None,
                "error": "Not enough statements",
                "reasoning": ["Not enough statements for logical analysis"]
            }
        
        # 전제와 결론 분리
        premises = sentences[:-1]
        conclusion = sentences[-1].replace("Therefore,", "").strip()
        
        # 기본 패턴 분석
        if len(premises) == 2:
            # 삼단논법 패턴
            if any(p.startswith(("All", "Every")) for p in premises):
                universal = next(p for p in premises if p.startswith(("All", "Every")))
                specific = next(p for p in premises if not p.startswith(("All", "Every")))
                
                # 모순 검사
                if "not" in conclusion or "immortal" in conclusion.lower():
                    # 결론이 전제와 모순되는 경우
                    return {
                        "status": "contradiction",
                        "result": False,
                        "error": None,
                        "reasoning": [
                            "The conclusion contradicts the universal premise",
                            "When we have a universal statement and a specific case,",
                            "the conclusion cannot contradict what logically follows"
                        ]
                    }
                else:
                    # 유효한 삼단논법
                    return {
                        "status": "valid",
                        "result": True,
                        "error": None,
                        "reasoning": [
                            "Valid syllogistic reasoning",
                            "The conclusion follows from the universal statement and the specific case"
                        ]
                    }
            
            # Modus Tollens 패턴
            elif any("If" in p for p in premises):
                if "not" in next(p for p in premises if "If" not in p):
                    if "not" in conclusion:
                        return {
                            "status": "valid",
                            "result": True,
                            "error": None,
                            "reasoning": [
                                "Valid modus tollens",
                                "If P implies Q, and not Q, then not P"
                            ]
                        }
        
        # 기본적으로 invalid 반환
        return {
            "status": "invalid",
            "result": False,
            "error": None,
            "reasoning": [
                "The conclusion does not necessarily follow from the premises",
                "The logical connection is not strong enough to guarantee the conclusion"
            ]
        }
        
    except Exception as e:
        return {
            "status": "error",
            "result": None,
            "error": str(e),
            "reasoning": [f"Error in logical analysis: {str(e)}"]
        }

def test_logical_argument(input_text: str):
    """논리 추론 테스트"""
    result = evaluate_logic({"input": input_text})
    print("Testing:", input_text)
    print("Status:", result["status"])
    print("Result:", result["result"])
    print("Reasoning:", result["reasoning"])
    if result.get("error"):
        print("Error:", result["error"])
    print()
    return result

# 테스트 실행
test_cases = [
    "All humans are mortal. Socrates is human. Therefore, Socrates is immortal.",
    "If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.",
    "All birds fly. Penguins are birds. Therefore, penguins fly."
]

for test_case in test_cases:
    test_logical_argument(test_case)

Testing: All humans are mortal. Socrates is human. Therefore, Socrates is immortal.
Status: contradiction
Result: False
Reasoning: ['The conclusion contradicts the universal premise', 'When we have a universal statement and a specific case,', 'the conclusion cannot contradict what logically follows']

Testing: If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.
Status: valid
Result: True
Reasoning: ['Valid modus tollens', 'If P implies Q, and not Q, then not P']

Testing: All birds fly. Penguins are birds. Therefore, penguins fly.
Status: valid
Result: True
Reasoning: ['Valid syllogistic reasoning', 'The conclusion follows from the universal statement and the specific case']



In [214]:
tests = [
    "If something is not impossible, then it is possible. This is not impossible. Therefore, this is possible.",
    "All mathematicians are logical. Some logical people are not creative. Therefore, some mathematicians are not creative.",
    "Only mammals have fur. This animal has no fur. Therefore, this animal is not a mammal.",
    "No reptiles are mammals. All snakes are reptiles. Therefore, no snakes are mammals.",
    "Everyone has someone who likes them. Alice likes nobody. Therefore, Alice is liked by someone.",
    "Every part of this machine is working. If every part is working, then the machine is functional. Therefore, this machine is functional.",
    "Nothing can be both completely solid and completely liquid at the same time. This substance is completely solid. Therefore, this substance is not completely liquid."
]

for test_case in tests:
    test_logical_argument(test_case)

Testing: If something is not impossible, then it is possible. This is not impossible. Therefore, this is possible.
Status: invalid
Result: False
Reasoning: ['The conclusion does not necessarily follow from the premises', 'The logical connection is not strong enough to guarantee the conclusion']

Testing: All mathematicians are logical. Some logical people are not creative. Therefore, some mathematicians are not creative.
Status: contradiction
Result: False
Reasoning: ['The conclusion contradicts the universal premise', 'When we have a universal statement and a specific case,', 'the conclusion cannot contradict what logically follows']

Testing: Only mammals have fur. This animal has no fur. Therefore, this animal is not a mammal.
Status: invalid
Result: False
Reasoning: ['The conclusion does not necessarily follow from the premises', 'The logical connection is not strong enough to guarantee the conclusion']

Testing: No reptiles are mammals. All snakes are reptiles. Therefore, no snake

In [223]:
from typing import TypedDict, List, Optional, Dict, Annotated
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

class LogicalState(TypedDict):
    """논리 추론 상태"""
    input: str
    messages: Annotated[List[HumanMessage | AIMessage], add_messages]
    status: str
    result: Optional[bool]
    reasoning: List[str]

def evaluate_logic(state: LogicalState) -> Dict:
    """논리적 타당성을 평가하는 함수"""
    input_text = state["input"]
    
    try:
        # 1. 전체 텍스트 분석
        analysis = analyze_text_flow(input_text)
        if analysis["error"]:
            return error_response(analysis["error"])
            
        # 2. 결론 식별
        conclusion = analysis["conclusion"]
        premises = analysis["premises"]
        
        # 3. 논리 구조 분석
        logical_structure = analyze_logical_structure(premises, conclusion)
        
        # 4. 논리적 추론
        return evaluate_reasoning(logical_structure)
        
    except Exception as e:
        return error_response(str(e))

def analyze_text_flow(text: str) -> Dict:
    """전체 텍스트의 논리 흐름 분석"""
    # LLM Prompt
    prompt = f"""Analyze this text and identify its logical structure:

Text: {text}

Please identify:
1. What is the main argument being made?
2. What are the supporting premises?
3. What is the conclusion (whether explicit or implicit)?
4. How do the statements connect logically?

Format your response as:
PREMISES: (list the premises)
CONCLUSION: (state the conclusion)
FLOW: (describe how the argument flows)
"""
    
    # LLM 응답 파싱
    # response = llm.generate(prompt)
    # 실제 구현에서는 LLM 응답을 파싱하여 아래 구조로 만듦
    
    return {
        "premises": [],  # 식별된 전제들
        "conclusion": "",  # 식별된 결론
        "flow": "",  # 논리 흐름 설명
        "error": None
    }

def analyze_logical_structure(premises: List[str], conclusion: str) -> Dict:
    """논리 구조 분석"""
    # LLM Prompt
    prompt = f"""Analyze these logical statements:

Premises:
{chr(10).join(f'- {p}' for p in premises)}

Conclusion:
{conclusion}

Identify:
1. Type of logical argument (e.g., deductive, inductive)
2. Logical patterns present (e.g., syllogism, modus ponens)
3. Key terms and their relationships
4. Any unstated assumptions

Format your response as a structured analysis.
"""
    
    # LLM 응답을 구조화된 형태로 변환
    return {
        "type": "",  # 논리 유형
        "patterns": [],  # 발견된 논리 패턴들
        "terms": {},  # 주요 용어와 관계
        "assumptions": []  # 암묵적 가정들
    }

def evaluate_reasoning(structure: Dict) -> Dict:
    """논리적 추론 평가"""
    # 논리 유형에 따른 평가
    if structure["type"] == "deductive":
        if check_deductive_validity(structure):
            return {
                "status": "valid",
                "result": True,
                "reasoning": ["Valid deductive reasoning", "Conclusion necessarily follows from premises"]
            }
    elif structure["type"] == "inductive":
        strength = evaluate_inductive_strength(structure)
        if strength > 0.7:  # 임계값
            return {
                "status": "valid",
                "result": True,
                "reasoning": ["Strong inductive reasoning", "Conclusion is well supported by premises"]
            }
            
    # 모순 검사
    if check_contradiction(structure):
        return {
            "status": "contradiction",
            "result": False,
            "reasoning": ["Premises and conclusion are contradictory"]
        }
        
    return {
        "status": "invalid",
        "result": False,
        "reasoning": ["The argument does not provide sufficient support for the conclusion"]
    }

def check_deductive_validity(structure: Dict) -> bool:
    """연역 논리의 타당성 검사"""
    patterns = structure["patterns"]
    terms = structure["terms"]
    
    # 각 논리 패턴 검사
    for pattern in patterns:
        if not is_pattern_valid(pattern, terms):
            return False
    return True

def evaluate_inductive_strength(structure: Dict) -> float:
    """귀납 논리의 강도 평가"""
    # 귀납 논리의 강도를 0~1 사이 값으로 평가
    evidence_strength = assess_evidence(structure["premises"])
    relevance = assess_relevance(structure["terms"])
    
    return (evidence_strength + relevance) / 2

def error_response(message: str) -> Dict:
    """에러 응답 생성"""
    return {
        "status": "error",
        "result": None,
        "error": message,
        "reasoning": [f"Error in analysis: {message}"]
    }

def test_logical_argument(input_text: str):
    """논리 추론 테스트"""
    result = evaluate_logic({"input": input_text})
    print("Testing:", input_text)
    print("Status:", result["status"])
    print("Result:", result["result"])
    print("Reasoning:", result["reasoning"])
    if result.get("error"):
        print("Error:", result["error"])
    print()
    return result

# 테스트 실행
test_cases = [
    "All humans are mortal. Socrates is human. Therefore, Socrates is immortal.",
    "If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.",
    "All birds fly. Penguins are birds. Therefore, penguins fly."
]

for test_case in test_cases:
    test_logical_argument(test_case)

Testing: All humans are mortal. Socrates is human. Therefore, Socrates is immortal.
Status: error
Result: None
Reasoning: ["Error in analysis: name 'check_contradiction' is not defined"]
Error: name 'check_contradiction' is not defined

Testing: If it rains, the ground is wet. The ground is not wet. Therefore, it did not rain.
Status: error
Result: None
Reasoning: ["Error in analysis: name 'check_contradiction' is not defined"]
Error: name 'check_contradiction' is not defined

Testing: All birds fly. Penguins are birds. Therefore, penguins fly.
Status: error
Result: None
Reasoning: ["Error in analysis: name 'check_contradiction' is not defined"]
Error: name 'check_contradiction' is not defined



In [221]:
test_cases = [
    """
    All scientific theories must be falsifiable. String theory makes no testable predictions. 
    Many physicists work on string theory despite this limitation. 
    The mathematics of string theory is elegant and complex. 
    Several alternative theories exist that make testable predictions. 
    Therefore, string theory is not a proper scientific theory.
    """,
    
    """
    All effective medical treatments must show results in controlled trials.
    Homeopathy has been tested in numerous controlled trials.
    These trials show no effect beyond placebo.
    Some people report feeling better after homeopathic treatment.
    Placebo effects can be powerful in certain conditions.
    Therefore, homeopathy is not an effective medical treatment.
    """,
    
    """
    All living organisms require energy to survive.
    Plants get their energy through photosynthesis.
    Deep sea creatures live where no light reaches.
    These creatures depend on chemical energy from thermal vents.
    Some bacteria can survive in extreme conditions.
    Therefore, not all living organisms depend on sunlight for energy.
    """
]

for test_case in test_cases:
    result = evaluate_logic({"input": test_case})
    print("Testing:", test_case.strip())
    print("Status:", result["status"])
    print("Result:", result["result"])
    print("Reasoning:", result["reasoning"])
    print("\n" + "="*80 + "\n")

Testing: All scientific theories must be falsifiable. String theory makes no testable predictions. 
    Many physicists work on string theory despite this limitation. 
    The mathematics of string theory is elegant and complex. 
    Several alternative theories exist that make testable predictions. 
    Therefore, string theory is not a proper scientific theory.
Status: contradiction
Result: False
Reasoning: ['The conclusion contradicts the universal premise', 'When we have a universal statement and a specific case,', 'the conclusion cannot contradict what logically follows']


Testing: All effective medical treatments must show results in controlled trials.
    Homeopathy has been tested in numerous controlled trials.
    These trials show no effect beyond placebo.
    Some people report feeling better after homeopathic treatment.
    Placebo effects can be powerful in certain conditions.
    Therefore, homeopathy is not an effective medical treatment.
Status: contradiction
Result: F

https://langchain-ai.github.io/langgraph/concepts/low_level/

In [232]:
from typing import TypedDict, List, Dict, Optional, Union
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END, START
from operator import add
from typing import Annotated
from langchain_openai import ChatOpenAI
import os

# LLM 초기화
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4"
)

# 1. 상태 정의
class LogicalAnalysisState(TypedDict):
    # 입력 텍스트
    input_text: str
    # 파악된 논리적 구문들
    logical_statements: List[str]
    # 각 구문의 진위 여부
    statement_validities: List[Dict[str, bool]]
    # 전체 신빙성 점수 (0-100)
    credibility_score: float
    # 분석 메시지들
    messages: Annotated[List[BaseMessage], add]

def analyze_context(state: LogicalAnalysisState) -> Dict:
    """텍스트를 읽고 문맥을 파악하여 논리적 구문들을 추출"""
    messages = [
        HumanMessage(content=f"""
                     take a deep breath and think step by step.
        다음 텍스트를 분석하여 주요 논리적 구문들을 추출해주세요:

        텍스트: {state['input_text']}

        다음 형식으로 응답해주세요:
        - 각 논리적 구문을 별도의 문장으로 나열
        - 전제와 결론 관계가 명확한 것들 위주로 추출
        - 암묵적 전제도 포함
        """)
    ]
    
    # LLM 호출
    response = llm.invoke(messages)
    
    logical_statements = [s.strip() for s in response.content.split('\n') if s.strip()]
    
    return {
        "logical_statements": logical_statements,
        "messages": [
            HumanMessage(content=state["input_text"]),
            AIMessage(content=f"파악된 논리적 구문들:\n{chr(10).join(logical_statements)}")
        ]
    }

def validate_statements(state: LogicalAnalysisState) -> Dict:
    """각 논리적 구문의 진위 여부 평가"""
    validities = []
    
    for statement in state["logical_statements"]:
        messages = [
            HumanMessage(content=f"""
                         take a deep breath and think step by step.
            다음 논리적 구문의 타당성을 평가해주세요:

            구문: {statement}

            다음을 고려하세요:
            1. 전제들이 결론을 논리적으로 뒷받침하는가?
            2. 숨겨진 가정이 있는가?
            3. 논리적 오류가 있는가?

            True/False로 응답하고 이유를 설명해주세요.
            """)
        ]
        
        # LLM 호출
        response = llm.invoke(messages)
        
        is_valid = "true" in response.content.lower()
        reason = response.content.split('\n')[1] if '\n' in response.content else response.content
        
        validities.append({
            "statement": statement,
            "is_valid": is_valid,
            "reason": reason
        })
    
    return {
        "statement_validities": validities,
        "messages": [AIMessage(content=f"논리 구문 평가 완료: {len(validities)}개 분석됨")]
    }

def calculate_credibility(state: LogicalAnalysisState) -> Dict:
    """전체 신빙성 점수 계산"""
    validities = state["statement_validities"]
    
    # 유효한 구문의 비율 계산
    valid_count = sum(1 for v in validities if v["is_valid"])
    total_count = len(validities)
    
    # 기본 점수 계산 (유효 비율 * 100)
    base_score = (valid_count / total_count) * 100 if total_count > 0 else 0
    
    # 가중치 적용 (예: 더 중요한 구문에 더 높은 가중치)
    weighted_score = base_score
    
    return {
        "credibility_score": weighted_score,
        "messages": [AIMessage(content=f"전체 신빙성 점수: {weighted_score:.1f}/100")]
    }

# 3. 그래프 구성
def create_analysis_graph():
    """논리 분석 그래프 생성"""
    graph = StateGraph(LogicalAnalysisState)
    
    # 노드 추가
    graph.add_node("context_analyzer", analyze_context)
    graph.add_node("statement_validator", validate_statements)
    graph.add_node("credibility_calculator", calculate_credibility)
    
    # 엣지 추가
    graph.add_edge(START, "context_analyzer")
    graph.add_edge("context_analyzer", "statement_validator")
    graph.add_edge("statement_validator", "credibility_calculator")
    graph.add_edge("credibility_calculator", END)
    
    return graph.compile()

def analyze_text_logic(text: str) -> Dict:
    """텍스트의 논리성 분석 실행"""
    graph = create_analysis_graph()
    
    # 초기 상태
    initial_state = {
        "input_text": text,
        "logical_statements": [],
        "statement_validities": [],
        "credibility_score": 0.0,
        "messages": []
    }
    
    # 그래프 실행
    result = graph.invoke(initial_state)
    
    return {
        "credibility_score": result["credibility_score"],
        "logical_statements": result["logical_statements"],
        "validities": result["statement_validities"],
        "analysis_log": [m.content for m in result["messages"]]
    }

# 실행 예시
if __name__ == "__main__":
    test_text = """
<Game>
## 핵심 규칙
1. 게임 조건
   - 게임 종료: 5라운드 도달 정답 수 3 미만 또는 5라운드 이하 정답 수 3
   - 라운드 정의:
	   - 문제가 출력되면 라운드 시작, total_round + 1
		   - 정답 시 다음 문제 출력 == 다음 라운드
		   - 첫 번째 오답 시 재시도 기회 기존 라운드
		   - 두 번째 오답 시 다음 문제 출력 == 라운드
   - 모든 출력은 JSON 형식

2. 카운터 규칙
   - total_round: 정답 또는 오답 2회시에만 +1
   - answer_count: 정답 시에만 +1
   - wrong_count: 오답 시 +1, 2회시 초기화
   - is_end: (total_round >= 5 || answer_count >= 3)

3. 변수 규칙
	 - current_answer: Base64로 인코딩된 정답

3. 정답 판정 규칙
   - **한글 입력은 무조건 오답 처리**
   - 반드시 영어 단어만 정답으로 인정
   - 입력된 영어 단어를 Base64로 인코딩하여 current_answer와 비교
   - 대소문자는 구분하지 않음 (see = SEE = See)
   - 입력된 모든 메시지는 정답 판정 실행

## 엄격한 규칙
1. 모든 출력은 반드시 위의 JSON 형식 준수
2. **한글 답변은 무조건 오답 처리**
3. 영어 단어만 Base64 인코딩하여 비교
4. 영어가 아닌 모든 입력은 오답으로 처리
5. 정확한 영어 단어만 정답으로 인정
6. 한글 뜻이나 설명은 정답으로 인정하지 않음
7. 라운드 증가는 정답 또는 오답 2회 시에만
8. 정답 수 증가는 정확한 정답 시에만
9. 모든 사용자 입력은 정답 판정 필수
10. 대소문자 구분 없는 정확한 문자열 매칭
11. 게임 종료 조건 도달 즉시 종료

## 상태별 출력 형식
1. 게임 시작 시
\```json
{
    "total_round": 0,
    "answer_count": 0,
    "current_answer": "첫문제_정답",
    "hint": ["첫문제_힌트"],
    "check_answer": false,
    "is_end": false,
    "message": "비슷하게 들리지만 다른 뜻을 가진 재미있는 영어 단어들을 배워볼까요?🎈 이번엔 {문제_제목}에 대해 배워요. \n\n문제: {첫문제}"
}
\```

2. 정답 입력 시
\```json
{
    "total_round": "{이전_라운드 + 1}",
    "answer_count": "{이전_정답수 + 1}",
    "current_answer": "{다음문제_정답}",
    "hint": ["{다음문제_힌트}"],
    "check_answer": true,
    "is_end": false,
    "message": "정답입니다! 🎉\n설명: {정답_설명}\n\n다음 문제: {다음_문제_문장}"
}
\```

3. 오답 1회 시
\```json
{
    "total_round": "{현재_라운드}",
    "answer_count": "{현재_정답수}",
    "current_answer": "{현재_정답}",
    "hint": ["{현재_힌트}"],
    "check_answer": false,
    "is_end": false,
    "message": "오답입니다! 😅\n\n힌트: {현재_힌트}"
}
\```

4. 오답 2회 시
\```json
{
    "total_round": "{이전_라운드 + 1}",
    "answer_count": "{현재_정답수}",
    "current_answer": "{다음문제_정답}",
    "hint": ["{다음문제_힌트}"],
    "check_answer": false,
    "is_end": false,
    "message": "오답입니다! 정답은 {현재_정답}입니다.\n설명: {현재_설명}\n\n다음 문제: {다음_문제_문장}"
}
\```

5. 게임 종료 시 (정답 3개)
\```json
{
    "total_round": "{최종_라운드}",
    "answer_count": 3,
    "current_answer": "",
    "hint": [],
    "check_answer": true,
    "is_end": true,
    "message": "동음이의어 마스터가 되었어요! 🏆\n총 {최종_라운드}라운드 중 3문제 정답!"
}
\```

6. 게임 종료 시 (5라운드 도달 and 정답 3개 미만)
\```json
{
    "total_round": 5,
    "answer_count": "{최종_정답수}",
    "current_answer": "",
    "hint": [],
    "check_answer": false,
    "is_end": true,
    "message": "괜찮아요! 힌트를 보고 다시 도전해봐요! 💪\n총 5라운드 중 {최종_정답수}문제 정답!"
}
\```

## wrong_count 처리 규칙
1. 오답 1회 시 (wrong_count === 1)
   - total_round 유지
   - 힌트 제공 메시지만 출력
   - 다음 문제로 넘어가지 않음

2. 오답 2회 시 (wrong_count === 2)
   - total_round 증가
   - 정답과 설명 제공
   - 다음 문제로 넘어감
   - wrong_count 초기화

## 게임 종료 시 추가 규칙
1. 마지막 라운드에서도 동일한 정답 판정 규칙 적용
2. 정답 카운트는 이전 상태값 유지
3. 5라운드 도달 시:
   - 정답 수가 0인 경우에도 정상적으로 처리
   - 마지막 입력이 오답이면 check_answer는 false
   - 실제 정답 수를 기준으로 메시지 출력
    """
    
    result = analyze_text_logic(test_text)
    
    print(f"신빙성 점수: {result['credibility_score']:.1f}/100")
    print("\n논리 구문들:")
    for stmt in result["logical_statements"]:
        print(f"- {stmt}")
    print("\n분석 로그:")
    for log in result["analysis_log"]:
        print(f"- {log}")

<>:157: SyntaxWarning: invalid escape sequence '\`'
<>:157: SyntaxWarning: invalid escape sequence '\`'
C:\Users\DNSOFT\AppData\Local\Temp\ipykernel_28408\2473134051.py:157: SyntaxWarning: invalid escape sequence '\`'
  test_text = """


신빙성 점수: 46.7/100

논리 구문들:
- - 게임은 총 5라운드로 구성되며, 각 라운드에서는 문제가 출력되고 사용자가 답변을 제출합니다.
- - 라운드는 문제가 출력될 때 시작되며, 정답을 맞추거나 오답을 두 번 제출하면 다음 라운드로 넘어갑니다.
- - 게임은 5라운드를 모두 진행하거나, 3개의 정답을 맞추면 종료됩니다.
- - 모든 게임 데이터는 JSON 형식으로 출력됩니다.
- - 정답을 맞추거나 오답을 두 번 제출할 때마다 total_round가 1씩 증가합니다.
- - 정답을 맞출 때마다 answer_count가 1씩 증가합니다.
- - 오답을 제출할 때마다 wrong_count가 1씩 증가하며, 오답을 두 번 제출하면 wrong_count는 초기화됩니다.
- - 게임이 종료되는 조건은 total_round가 5 이상이거나 answer_count가 3 이상일 때입니다.
- - 정답은 Base64로 인코딩된 영어 단어로, 대소문자를 구분하지 않습니다.
- - 한글 입력은 무조건 오답으로 처리됩니다.
- - 입력된 영어 단어를 Base64로 인코딩하여 정답과 비교합니다.
- - 입력된 모든 메시지는 정답 판정을 실행합니다.
- - 게임 종료 시 마지막 라운드의 정답 판정 규칙이 적용되며, 정답 카운트는 이전 상태값을 유지합니다.
- - 5라운드에 도달했을 때, 정답 수가 0인 경우에도 정상적으로 처리하며, 마지막 입력이 오답이면 check_answer는 false로 설정됩니다.
- - 게임 종료 메시지는 실제 정답 수를 기준으로 출력됩니다.

분석 로그:
- 
<Game>
## 핵심 규칙
1. 게임 조건
   - 게임 종료: 5라운드 도달 정답 수 3 미만 또는 5라운드 이하 정답 수 3
   - 라운드 정의:
	   - 문제가 출력되면 라운드 시작, total_round + 1
		   - 정답 시 다음 문제 출력 == 다음 라운드
		   - 첫 번째 오답 시 재시도 기회 기존 라운드
		   - 두 번째 오답 시 다음 문제 출력 == 라운드